Meta-Llama-3.1-8B-Instruct-GGUF/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf

In [15]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import time
from typing import List, Dict, Optional

In [16]:
data = pd.read_json("./News_Category_Dataset_v3.json", lines=True)

for col in data.columns:
    print(col)
    
# Filtra le notizie con categoria "WEIRD NEWS"
news = data[data['category'] == 'WEIRD NEWS']

# Mostra le prime righe per verifica
print(news.head())

# Opzionale: Salvare le weird news in un nuovo file JSON
news.to_json("weird_news.json", orient="records", lines=True)

news.to_csv("Weird_News.csv", index=False)

link
headline
category
short_description
authors
date
                                                  link  \
29   https://www.huffpost.com/entry/french-spiderma...   
38   https://www.huffpost.com/entry/african-land-sn...   
154  https://www.huffpost.com/entry/fisherman-prehi...   
159  https://www.huffpost.com/entry/canberra-kangar...   
239  https://www.huffpost.com/entry/wisconsin-polic...   

                                              headline    category  \
29   'French Spider-Man' Celebrates 60th Birthday W...  WEIRD NEWS   
38   German Customs Officials Follow Trail Of Slime...  WEIRD NEWS   
154  A Fisherman Thought He Caught A Rock. He Snagg...  WEIRD NEWS   
159  FINISH HIM: Fighting Kangaroo Pulls Wild Move ...  WEIRD NEWS   
239  Feathers Fly As Wild Turkey Dodges Cops In Sla...  WEIRD NEWS   

                                     short_description       authors  \
29   Alain Robert, known as the "French Spider-Man,...  Ben Blanchet   
38   “Never in the history of th

In [17]:
for index, row in news.head(5).iterrows():
    links = row['link']

In [18]:
def extract_article_text(links: str) -> Optional[str]:
    """
    Estrae il testo principale di un articolo da un URL.
    
    Args:
        url (str): L'URL dell'articolo da analizzare
        
    Returns:
        Optional[str]: Il testo dell'articolo o None in caso di errore
    """
    try:
        # Aggiungi headers per evitare blocchi
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(links, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Rimuovi elementi non necessari
        for elem in soup.find_all(['script', 'style', 'nav', 'header', 'footer']):
            elem.decompose()
            
        # Cerca il contenuto principale dell'articolo
        # Questo è un approccio generico, potrebbe dover essere personalizzato per siti specifici
        article_content = None

        # Prova diverse strategie comuni per trovare il contenuto
        selectors = [
            'article',
            '[class*="article"]',
            '[class*="content"]',
            'main',
            '.post-content'
        ]
        
        for selector in selectors:
            content = soup.select_one(selector)
            if content:
                article_content = content
                break
                
        if not article_content:
            # Se non troviamo contenuti specifici, prendiamo tutti i paragrafi
            article_content = soup.find_all('p')
            
        # Estrai il testo
        if isinstance(article_content, list):
            text = ' '.join(p.get_text().strip() for p in article_content)
        else:
            text = article_content.get_text().strip()
            
        # Pulizia base del testo
        text = ' '.join(text.split())
        return text
        
    except Exception as e:
        print(f"Errore nell'estrazione del testo da {links}: {e}")
        return None

def analyze_articles(links: List[str]) -> List[Dict]:
    """
    Analizza il sentiment di una lista di articoli.
    
    Args:
        urls (List[str]): Lista di URL degli articoli da analizzare
        
    Returns:
        List[Dict]: Lista di risultati con URL e sentiment
    """
    results = []
    
    def process_article(links: str) -> Dict:
        try:
            # Estrai il testo dell'articolo
            article_text = extract_article_text(links)
            if not article_text:
                return {"url": links, "status": "error", "error": "Impossibile estrarre il testo"}
            
            # Analizza il sentiment
            sentiment_result = analyze_sentiment(article_text)
            if not sentiment_result:
                return {"url": links, "status": "error", "error": "Errore nell'analisi del sentiment"}
                
            # Estrai il valore numerico dal completamento
            sentiment_value = int(sentiment_result['choices'][0]['text'].strip())
            
            return {
                "url": links,
                "status": "success",
                "sentiment": sentiment_value,
                "text_length": len(article_text)
            }
            
        except Exception as e:
            return {"url": links, "status": "error", "error": str(e)}
    
    # Processa gli articoli in parallelo
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(process_article, links))
    
    return results

# Esempio di utilizzo
if __name__ == "__main__":
    # Lista di URL da analizzare
    
    # Analizza gli articoli
    results = analyze_articles(links)
    
    # Stampa i risultati
    for result in results:
        print(f"\nAnalisi per {result['url']}:")
        if result['status'] == 'success':
            sentiment_map = {1: "Positivo", 0: "Neutro", -1: "Negativo"}
            print(f"Sentiment: {sentiment_map[result['sentiment']]}")
            print(f"Lunghezza testo: {result['text_length']} caratteri")
        else:
            print(f"Errore: {result['error']}")

Errore nell'estrazione del testo da h: Invalid URL 'h': No scheme supplied. Perhaps you meant https://h?
Errore nell'estrazione del testo da t: Invalid URL 't': No scheme supplied. Perhaps you meant https://t?
Errore nell'estrazione del testo da p: Invalid URL 'p': No scheme supplied. Perhaps you meant https://p?
Errore nell'estrazione del testo da :: No connection adapters were found for ':'
Errore nell'estrazione del testo da t: Invalid URL 't': No scheme supplied. Perhaps you meant https://t?
Errore nell'estrazione del testo da s: Invalid URL 's': No scheme supplied. Perhaps you meant https://s?
Errore nell'estrazione del testo da w: Invalid URL 'w': No scheme supplied. Perhaps you meant https://w?
Errore nell'estrazione del testo da w: Invalid URL 'w': No scheme supplied. Perhaps you meant https://w?
Errore nell'estrazione del testo da /: Invalid URL '/': No scheme supplied. Perhaps you meant https:///?
Errore nell'estrazione del testo da /: Invalid URL '/': No scheme supplied. Per

In [19]:
def analyze_sentiment(text, api_url="http://localhost:1234/api/v0/completions"): 
    """
    Analizza il sentiment di un testo usando un modello LLM locale
    
    Args:
        text (str): Il testo da analizzare
        api_url (str): URL dell'API locale
    
    Returns:
        dict: La risposta dell'API contenente il completamento
    """
    
    headers = {
        "Content-Type": "application/json"
    }
    
    # Costruisco il prompt per l'analisi del sentiment
    prompt = f"""Analizza il sentiment del seguente testo e classifica come 1 se positivo, -1 se negativo o 0 se neutro.
                 Rispondi solo con uno di questi tre numero.
                 Testo: '{text}'
              """
    
    data = {
        "model": "Meta-Llama-3.1-8B-Instruct-4bit",
        "prompt": prompt,
        "temperature": 0.7,
        "max_tokens": 10,
        "stream": False,
        "stop": "\n"
    }
    
    try:
        response = requests.post(api_url, headers=headers, data=json.dumps(data))
        response.raise_for_status()  # Solleva un'eccezione per errori HTTP
        return response.json()
    
    except requests.exceptions.RequestException as e:
        print(f"Errore nella chiamata API: {e}")
        return None


In [20]:
for index, row in news.head(20).iterrows():
    title = row['headline']
    short_description = row['short_description']
    news_id = index
    date = row['date']
   
    text_to_analyze = f"{title}. {short_description}"
   
    # Analizza il sentiment
    api_response = analyze_sentiment(text_to_analyze)
   
    # Debug: stampa la risposta completa
    print(f"\nAnalisi per notizia {news_id}:")
    print(f"Testo: '{text_to_analyze}'")
    print(f"Risposta API: {api_response}")
   
    # Estrai il punteggio
    sentiment_score = extract_sentiment_score(api_response)
   
    # Aggiungi il risultato
    results.append({
        "ID": news_id,
        "Date": date,
        "Sentiment": sentiment_score if sentiment_score is not None else -1  # Usa -1 per indicare errore
    })



Analisi per notizia 29:
Testo: ''French Spider-Man' Celebrates 60th Birthday With A Massive Skyscraper Climb. Alain Robert, known as the "French Spider-Man," has climbed the Burj Khalifa and the Empire State Building.'
Risposta API: {'id': 'cmpl-ukev20i35blhxlmi5v1n6s', 'object': 'text_completion', 'created': 1738227117, 'model': 'meta-llama-3.1-8b-instruct', 'choices': [{'index': 0, 'text': "'' Risposta: 0", 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 93, 'completion_tokens': 5, 'total_tokens': 98}, 'stats': {'tokens_per_second': 19.880043288661724, 'time_to_first_token': 1.075, 'generation_time': 0.303, 'stop_reason': 'stopStringFound'}, 'model_info': {'arch': 'llama', 'quant': '4bit', 'format': 'safetensors', 'context_length': 4096}, 'runtime': {'name': 'mlx-llm-mac-arm64-apple-metal-advsimd', 'version': '0.3.0', 'supported_formats': ['safetensors']}}


NameError: name 'extract_sentiment_score' is not defined

In [ ]:
testi = [
        "Questo prodotto è fantastico, lo consiglio vivamente!",
        "Non mi è piaciuto per niente, pessimo servizio.",
        "Il pacco è arrivato in orario come previsto."
    ]
    
for testo in testi:
    risultato = analyze_sentiment(testo)
    if risultato:
        print(f"\nTesto: {testo}")
        print(f"Sentiment: {risultato['choices'][0]['text'].strip()}")